# GPU加速计算

---

在Faiss中，由于存在大量简单但计算密集型的计算，因此GPU加速计算是非常有必要的。Faiss提供了多种GPU加速计算的方式：
1. 使用单GPU实现的GPU加速计算
2. 使用多GPU加速计算

---

在GPU上完成计算之后，我们可以需要将构建成功的索引传输回CPU，并最终通过`write_index`方法将索引保存到磁盘上。*GPU不允许直接将索引保存到磁盘上*。

### 第一种方式：使用单张GPU运算(`index_cpu_to_gpu`)
**所需参数：**
1. GPU实例资源
2. GPU编号（如果电脑上安装了多张GPU的话）
3. 需要转移运算的Index

In [15]:
# 检查电脑中的GPU配置
import faiss
import numpy as np
import time

# 获取可用 GPU 的数量
num_gpus = faiss.get_num_gpus()
print(f"GPU的数量是:{num_gpus}")

GPU的数量是:1


GPU的编号是按顺序进行标号的：0、1、2、3...

In [16]:
# 固定随机种子
np.random.seed(42)

# 创建全局数据与查询向量
dim = 256
data = np.random.rand(100000, dim)
query = np.random.rand(2, dim)

# 验证索引是否被转移到 GPU 上
index_cpu = faiss.IndexFlatL2(dim)
print(f"未进行转移的Index:{index_cpu}")

# 获得 GPU 运算资源
src = faiss.StandardGpuResources()

# 将索引转移至 GPU 上，返回一个Index对象
index_gpu = faiss.index_cpu_to_gpu(src, 0, index_cpu)  # GPU 实例资源，设备编号，需要转移的索引
print(f"已进行转移的Index:{index_gpu}")


未进行转移的Index:<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001C9E6C22270> >
已进行转移的Index:<faiss.swigfaiss.GpuIndexFlat; proxy of <Swig Object of type 'faiss::gpu::GpuIndexFlat *' at 0x000001C9E6C22420> >


可以发现index_gpu返回的对象已经包含了gpu，说明转移成功！

In [17]:
# 使用上一章学的IndexIVFPQ来检测GPU的运算效率

def test():
    # 初始化CPU类index
    quantizer = faiss.IndexFlatL2(dim)
    index_cpu = faiss.IndexIVFPQ(quantizer, dim, 500, 8, 8) # 2^8 = 256

    # 转化至GPU类index
    src = faiss.StandardGpuResources()
    index_gpu = faiss.index_cpu_to_gpu(src, 0, index_cpu)

    # 创建完整索引并添加数据
    index_gpu.train(data)
    assert index_gpu.is_trained
    index_gpu.add(data)
    start_time_test = time.time()
    index_gpu.nprobe = 1
    D, I = index_gpu.search(query, k=2)

    # 返回结果
    print(f"查询所需时间为:{time.time()-start_time_test:.4f}s")
    print(f"最短距离为：{D}")
    print(f"最短距离索引为：{I}", end='\n' + '*' * 9 + '\n')

    # 将index转化为 CPU 用于存储到本地，否则会报错
    index_cpu = faiss.index_gpu_to_cpu(index_gpu)
    faiss.write_index(index_cpu, "GPU_to_CPU.faiss")

In [18]:
# 允许主程序
if __name__ == "__main__":
    total_start_time = time.time()
    test()
    print(f"程序总运行时间是:{time.time() - total_start_time}")

查询所需时间为:0.0010s
最短距离为：[[21.64148  22.322227]
 [24.571217 24.57715 ]]
最短距离索引为：[[ 4344  7131]
 [84326 24785]]
*********
程序总运行时间是:1.3415923118591309


### 最后，让我们比较一下使用了GPU加速和没使用GPU加速在相同数据下的效率差异☺️

**未使用GPU加速时的程序用时：**<br/>

---

<img src="./GPU_simple.png" width="307">

**使用GPU加速时的程序用时：**<br/>

---

<img src="./GPU_accelerate.png" width="270">

我们可以发现，程序运行时间大大减少了，GPU万岁